In [ ]:
import numpy as np

class MyModel:
    """
    距離行列 dist_matrix, パラメータ beta, 人口密度ベクトル h などを
    コンストラクタで受け取り、一度だけ計算した w, U^L, U^F を
    以後のメソッドで使い回すクラス。
    """

    def __init__(self, dist_matrix, beta, h, J_L, J_F):
        """
        dist_matrix: ndarray of shape (|I|, |J|)
        beta: float
        h: ndarray of shape (|I|,)  (人口密度)
        J_L, J_F: ndarray of shape (|J|,)  (0/1 ベクトル)
                  施設が選ばれている場合に1，選ばれていない場合0
                  例: J_L は L施設セットの指示, J_F は F施設セットの指示
        """
        self.dist_matrix = dist_matrix       # shape (|I|, |J|)
        self.beta = beta
        self.h = h                           # shape (|I|,)
        self.J_L = J_L                       # shape (|J|,)
        self.J_F = J_F                       # shape (|J|,)

        # w[i,j] = exp(-beta * d_ij)
        self.w = np.exp(-beta * dist_matrix)  # shape (|I|, |J|)
        
        # U^L_i = sum_{j in J_L} w_{ij}
        self.U_L = (self.w * self.J_L).sum(axis=1)  # shape (|I|,)
        # U^F_i = sum_{j in J_F} w_{ij}
        self.U_F = (self.w * self.J_F).sum(axis=1)  # shape (|I|,)

    def cal_sum_w_ij(self, X):
        """
        X: ndarray of shape (|J|,) の0/1ベクトル (あるいは 0~1 の連続変数でも可)
        戻り値: shape (|I|,)
            各 i に対して sum_{j in X} w_{ij} (X_j=1なら j を含む) の計算
        """
        # self.w: (|I|, |J|), X: (|J|,)
        # X を w に掛けると、axis=1 (列方向)で掛け算したのを合計すれば良い
        # ただし numpy のブロードキャストを使うか、dotを使うか
        # ここでは dot 例:
        #   sum_{j} w_{i,j} * X_j -> これは (|I|,) になる
        sum_w_ij = np.dot(self.w, X)  # (|I|,)  ( w_{i,j} × X_j を jについて合計 )
        return sum_w_ij

    def cal_sum_w_ij_XY(self, X, Y):
        """
        X, Y: ndarray of shape (|J|,)
              XとYの合集合について w_{ij} を合計 (X ∪ Y で1になっている列)
        戻り値: shape (|I|,)
            各 i に対して sum_{j in X ∪ Y} w_{ij} を計算
        """
        # X ∪ Y は or (論理和) で求められる
        X_union_Y = np.logical_or(X, Y).astype(int)
        sum_w_ij_XY = self.cal_sum_w_ij(X_union_Y)
        return sum_w_ij_XY

    def cal_f_i_Y(self, X, Y):
        """
        f_{i, Y}(X) = (U^L_i + sum_{j} w_{i,j} X_j) / (U^L_i + U^F_i + sum_{j} w_{i,j} (X_j + Y_j))
        戻り値: shape (|I|,) 各 i について f_{i,Y}(X)
        """
        sum_w_i_X = self.cal_sum_w_ij(X)            # Σ_j w_{ij} X_j
        sum_w_i_XY = self.cal_sum_w_ij_XY(X, Y)     # Σ_j w_{ij} (X_j + Y_j)
        numerator = self.U_L + sum_w_i_X           # 分子
        denominator = self.U_L + self.U_F + sum_w_i_XY
        f_i_Y = numerator / denominator
        return f_i_Y

    def cal_L_Y(self, X, Y):
        """
        L_Y(X) = Σ_{i in I} [ h_i * f_{i,Y}(X) ]
        戻り値: float
        """
        f_i_Y = self.cal_f_i_Y(X, Y)   # shape (|I|,)
        L_Y_X = np.dot(self.h, f_i_Y)  # Σ_i h_i * f_{i,Y}(X)
        return L_Y_X

    def cal_rho_k(self, k, X, Y):
        """
        rho_k = L_Y(X ∪ {k}) - L_Y(X)
        ただし、引数 k は「ひとつの施設 j=k を選択した 0/1ベクトル」とするか、
        あるいは X に既に組み込み済みなら Xのk番目を1にして計算。
        
        例: k_vec = np.zeros_like(X); k_vec[k_idx] = 1
             X_union_k = np.logical_or(X, k_vec)
        """
        # ここでは k も X, Y と同様に (|J|,) 0/1ベクトルとして扱う想定
        X_union_k = np.logical_or(X, k).astype(int)

        val_union = self.cal_L_Y(X_union_k, Y)
        val_X     = self.cal_L_Y(X, Y)
        rho = val_union - val_X
        return rho
